In [1]:
# Import required packages
import gc
import os
import shutil

import numpy as np
import progressbar as pb
import sklearn.utils as sku

import Config as conf
import CSV as csv

In [2]:
# Import & shuffle CSV data
xx, yy = csv.getCSV()
xx, yy = sku.shuffle(xx, yy, random_state=0)
xo, yo = xx, yy

Importing CSV files...
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_001.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_002.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_003.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_004.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_005.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_006.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_007.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_008.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_009.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_010.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_011.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_012.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_013.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_014.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_015.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_016.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85/csi_017.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH85

In [3]:
# Import Keras
from keras.applications.vgg16 import VGG16
import keras.callbacks as kc
import keras.layers as kl
import keras.models as km
import keras.optimizers as ko
import keras.utils as ku

Using TensorFlow backend.


In [5]:
# Setup Keras VGG16 Model
print("GC Collected:", gc.collect())
adam = ko.Adam(lr=conf.LEARNING_RATE, amsgrad=True)
lstm = kl.LSTM(
    conf.N_HIDDEN,
    unit_forget_bias=True,
    input_shape=(conf.WINDOW_SIZE, conf.N_COLUMNS))
lstm.add_loss(1e-8)
model = km.Sequential()
model.add(lstm)
model.add(
    kl.Dense(
        conf.USE_NOACTIVITY and conf.N_CLASSES or conf.N_VALID_CLASSES,
    activation="softmax"))
model.compile(
    loss="categorical_crossentropy",
    optimizer=adam,
    metrics=["accuracy"]
)
model.summary()

GC Collected: 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 540)               1362960   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1082      
Total params: 1,364,042
Trainable params: 1,364,042
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Check output directory and prepare tensorboard
conf.BATCH_SIZE = 48
outputDir = conf.OUTPUT_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                                    conf.N_FILTERS, "")
if os.path.exists(outputDir):
    shutil.rmtree(outputDir)
os.makedirs(outputDir)
logDir = conf.LOG_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                              conf.N_FILTERS, "")
if os.path.exists(logDir):
    shutil.rmtree(logDir)
os.makedirs(logDir)
tensorboard = kc.TensorBoard(
    log_dir=logDir,
    histogram_freq=0,
    batch_size=conf.BATCH_SIZE,
    write_graph=True,
    write_grads=True,
    write_images=True,
    update_freq=10)
print(
    "If you have tensorboard in this environment, you can type below to see the result in tensorboard:"
)
print("    tensorboard --logdir=" + logDir)
print("Keras checkpoints and final result will be saved in here:")
print("    " + outputDir)

If you have tensorboard in this environment, you can type below to see the result in tensorboard:
    tensorboard --logdir=./Log_LR0.002_BATCH48_GWRRATE80/
Keras checkpoints and final result will be saved in here:
    ./Output_LR0.002_BATCH48_GWRRATE80/


In [8]:
# Modify activity:non-activity ratio
conf.ANOA_RATIO = 3
if conf.USE_NOACTIVITY:
    xx = np.empty([0, conf.WINDOW_SIZE, conf.N_COLUMNS], float)
    yy = np.empty([0, conf.N_CLASSES], float)
    with pb.ProgressBar(max_value=len(yo)) as bar:
        bar.update(0)
        for i in range(len(yo)):
            if yo[i, 0] == 0:
                yy = np.concatenate((yy, yo[i, :][np.newaxis, ...]), axis=0)
                xx = np.concatenate((xx, xo[i, :][np.newaxis, ...]), axis=0)
            bar.update(i)
    print("Found Activity Y:", len(yy))
    noacmx = int(len(yy) * conf.ANOA_RATIO)
    print("Will find", noacmx, "Non-Activity windows")
    noac = 0
    with pb.ProgressBar(max_value=noacmx) as bar:
        bar.update(0)
        for i in range(len(yo)):
            if noac > noacmx:
                break
            if not yo[i, 0] == 0:
                yy = np.concatenate((yy, yo[i, :][np.newaxis, ...]), axis=0)
                xx = np.concatenate((xx, xo[i, :][np.newaxis, ...]), axis=0)
                bar.update(noac)
                noac += 1
    print("Input Ratio Modified, Final Shape: [xx]", xx.shape, "[yy]", yy.shape)

100% (4158 of 4158) |####################| Elapsed Time: 0:00:35 Time:  0:00:35
N/A% (0 of 627) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

Found Activity Y: 209
Will find 627 Non-Activity windows


100% (627 of 627) |######################| Elapsed Time: 0:05:45 Time:  0:05:45


Input Ratio Modified, Final Shape: [xx] (837, 3760, 90) [yy] (837, 2)


In [ ]:
# Run KFold
conf.KFOLD = 8
conf.BATCH_SIZE = 48
conf.CP_PERIOD = 1
conf.N_EPOCH = 256
xx, yy = sku.shuffle(xx, yy, random_state=0)
for i in range(conf.KFOLD):
    # Run Garbage Collector
    print("Grabage Collecting Time! =>", gc.collect())
    
    # Roll the data
    xx = np.roll(xx, int(len(xx) / conf.KFOLD), axis=0)
    yy = np.roll(yy, int(len(yy) / conf.KFOLD), axis=0)

    # Data separation
    xTrain = xx[int(len(xx) / conf.KFOLD):]
    yTrain = yy[int(len(yy) / conf.KFOLD):]
    xEval = xx[:int(len(xx) / conf.KFOLD)]
    yEval = yy[:int(len(yy) / conf.KFOLD)]

    if not conf.USE_NOACTIVITY:
        # Remove NoActivity from ys
        yTrain = yTrain[:, 1:]
        yEval = yEval[:, 1:]

        # If there exists only one action, convert Y to binary form
        if yEval.shape[1] == 1:
            yTrain = ku.to_categorical(yTrain)
            yEval = ku.to_categorical(yEval)

    # Setup Keras Checkpoint
    checkpoint = kc.ModelCheckpoint(
        outputDir + "Checkpoint_K" + str(i + 1) + "_EPOCH{epoch}_ACC{val_acc:.6f}.h5", period=conf.CP_PERIOD)

    # Fit model (learn)
    print(
        str(i + 1) + " th fitting started. Endpoint is " + str(conf.KFOLD) +
        " th.")

    model.fit(
        xTrain,
        yTrain,
        batch_size=conf.BATCH_SIZE,
        epochs=conf.N_EPOCH,
        verbose=1,
        callbacks=[tensorboard, checkpoint],
        validation_data=(xEval, yEval))  # , validation_freq=2)
print("Epoch completed!")

Grabage Collecting Time! => 18
1 th fitting started. Endpoint is 8 th.
Instructions for updating:
Use tf.cast instead.
Train on 733 samples, validate on 104 samples
Epoch 1/256


In [ ]:
# Saving model
print("Saving model & model information...")
modelYML = model.to_yaml()
with open(outputDir + "model.yml", "w") as yml:
    yml.write(modelYML)
modelJSON = model.to_json()
with open(outputDir + "model.json", "w") as json:
    json.write(modelJSON)
model.save(outputDir + "model.h5")
print('Model saved! Congratulations! You finished all processes of ML!')

In [ ]:
# Finished